In [30]:
# l=[pd.NA for i in range(10)]
# import numpy as np
# s=np.random.randint(1,5,(5,)).tolist()
# x=np.random.randint(1,5,(5,)).tolist()
# l=s+l+x+l
# l=pd.Series(l)
# l=pd.to_numeric(l)

# l=l.interpolate(method='linear', limit=3, limit_direction='both', limit_area='inside')
# l

In [31]:
import pandas as pd

def clean(df, limit_days, feature_threshold):
    df['Timestamp']=pd.to_datetime(df['Timestamp'])
    df=df.set_index('Timestamp')
    start_date=df.index[0]
    end_date=df.index[-1]
    full_index=pd.date_range(start_date,end_date, freq='1h')
    df=df.reindex(full_index)
    limit_hours=24*limit_days
    df=df.interpolate(method='linear', limit=limit_hours, limit_direction='both', limit_area='inside', axis=0)
    df=df.dropna(axis=0, how='all')
    num_rows=df.shape[0]
    feature_present_fraction=df.notnull().sum(axis=0)/num_rows
    df = df.drop(columns=feature_present_fraction[feature_present_fraction < feature_threshold].index)
    df= df.dropna(axis=0, how='any')
    return df

In [32]:
df=pd.read_csv(r"/home/rishi/ML Projects/Air Pollution/CPCB/sites_comb/site_103_CRRI_Mathura_Road_Delhi_IMD_15Min.csv")
df.shape

(78888, 25)

In [33]:
x=clean(df,limit_days=7, feature_threshold=0.7)

In [34]:
x.shape
# x.to_csv("x_v1.csv")

(71147, 8)

In [35]:
x.shape

(71147, 8)

In [36]:
def num_gaps(df):
    time_diffs = df.index.to_series().diff()
    expected_freq = pd.Timedelta(hours=1)
    is_gap = time_diffs > expected_freq
    group_ids = is_gap.cumsum()

    # Calculate duration for each group using agg
    group_durations = df.groupby(group_ids).apply(
        lambda g: (g.index.max() - g.index.min()).total_seconds() / (24 * 3600),
    )
    
    value_counts = group_durations.value_counts().sort_index()
    
    return value_counts, sum(value_counts)

In [37]:
g=num_gaps(x)
g

(7.125000       1
 444.750000     1
 1173.416667    1
 1339.000000    1
 Name: count, dtype: int64,
 4)

In [38]:
import os

def save_contiguous_series(df, file_name):
    # Create folder with the file_name
    os.makedirs(file_name, exist_ok=True)
    
    # Identify gaps and create group IDs
    time_diffs = df.index.to_series().diff()
    expected_freq = pd.Timedelta(hours=1)
    is_gap = time_diffs > expected_freq
    group_ids = is_gap.cumsum()
    
    # Save each contiguous group
    for group_id, group_df in df.groupby(group_ids):
        start_time = group_df.index.min().strftime('%Y%m%d_%H%M%S')
        end_time = group_df.index.max().strftime('%Y%m%d_%H%M%S')
        
        # Reset index and put timestamp in 'Timestamp' column
        group_df_reset = group_df.reset_index()
        group_df_reset.rename(columns={'index': 'Timestamp'}, inplace=True)
        
        output_filename = f"{file_name}/{file_name}_{start_time}_to_{end_time}.csv"
        group_df_reset.to_csv(output_filename, index=False)
        
    print(f"Saved {len(df.groupby(group_ids))} contiguous series to folder '{file_name}/'")
    
    return len(df.groupby(group_ids))

In [39]:
# save_contiguous_series(x, "temp_split")

In [40]:
from concurrent.futures import ProcessPoolExecutor
import glob

def process_site(file_path):
    """Process a single site: clean data and count gaps"""
    # Extract site name from file path
    site_name = os.path.basename(file_path).replace('.csv', '')
    
    # Read dataset
    df = pd.read_csv(file_path)
    
    # Clean the data
    df_cleaned = clean(df, limit_days=7, feature_threshold=0.7)
    
    # Count gaps
    gap_counts, total_gaps = num_gaps(df_cleaned)
    
    # # Create cleaned folder structure
    # cleaned_folder = 'cleaned'
    # site_folder = os.path.join(cleaned_folder, site_name)
    # os.makedirs(site_folder, exist_ok=True)
    
    # # Save cleaned data
    # output_path = os.path.join(site_folder, f"{site_name}_cleaned.csv")
    # df_cleaned_reset = df_cleaned.reset_index()
    # df_cleaned_reset.rename(columns={'index': 'Timestamp'}, inplace=True)
    # df_cleaned_reset.to_csv(output_path, index=False)
    
    print(f"Processed {site_name}: {total_gaps} gaps")
    
    return site_name, (total_gaps, gap_counts)

# Get all CSV files from sites_comb folder
sites_comb_path = 'sites_comb/*.csv'
csv_files = glob.glob(sites_comb_path)

print(f"Found {len(csv_files)} datasets to process")

# Process all sites using ProcessPoolExecutor
with ProcessPoolExecutor() as executor:
    results = executor.map(process_site, csv_files)

# Create dictionary of gaps per site
gaps_per_site = dict(results)

print(f"\nProcessing complete! Total sites processed: {len(gaps_per_site)}")
print(f"Gaps per site dictionary created with {len(gaps_per_site)} entries")


Found 564 datasets to process
Processed site_5410_MIT-Kothrud_Pune_IITM_15Min: 4 gaps
Processed site_5042_Shasthri_Nagar_Ratlam_IPCA_Lab_15Min: 5 gaps
Processed site_5055_General_Hospital_Mandikhera(Nuh)_HSPCB_15Min: 1 gaps
Processed site_5414_Haji_Colony_Raichur_KSPCB_15Min: 12 gaps
Processed site_1422_Dwarka-Sector_8_Delhi_DPCC__15Min: 2 gaps
Processed site_1431_Patparganj_Delhi_DPCC_15Min: 4 gaps
Processed site_5071_Phase-1_GIDC_Vapi_GPCB_15Min: 13 gaps
Processed site_5045_Arya_Nagar_Bahadurgarh_HSPCB_15Min: 2 gapsProcessed site_5114_Mahape_Navi_Mumbai_MPCB_15Min: 5 gaps

Processed site_5334_Polayathode_Kollam_Kerala_PCB_15Min: 1 gaps
Processed site_5065_GIDC_Ankleshwar_GPCB_15Min: 6 gaps
Processed site_147_MD_University_Rohtak_HSPCB_15Min: 11 gaps
Processed site_5238_Ballygunge_Kolkata_WBPCB_15Min: 1 gapsProcessed site_5468_IESD_Banaras_Hindu_University_Varanasi_UPPCB_15Min: 1 gapsProcessed site_5472_Madan_Mohan_Malaviya_University_of_Technology_Gorakhpur_UPPCB_15Min: 1 gaps
Proces

In [41]:
gaps_per_site

{'site_1431_Patparganj_Delhi_DPCC_15Min': (4,
  372.708333    1
  672.083333    1
  771.708333    1
  894.500000    1
  Name: count, dtype: int64),
 'site_5042_Shasthri_Nagar_Ratlam_IPCA_Lab_15Min': (5,
  14.166667      1
  33.041667      1
  133.083333     1
  378.916667     1
  1892.333333    1
  Name: count, dtype: int64),
 'site_5468_IESD_Banaras_Hindu_University_Varanasi_UPPCB_15Min': (1,
  3286.958333    1
  Name: count, dtype: int64),
 'site_5334_Polayathode_Kollam_Kerala_PCB_15Min': (1,
  3286.958333    1
  Name: count, dtype: int64),
 'site_5612_SIPCOT_Phase-1_Hosur_TNPCB_15Min': (1,
  3286.958333    1
  Name: count, dtype: int64),
 'site_5452_Raikhad_Ahmedabad_IITM_15Min': (1,
  3286.958333    1
  Name: count, dtype: int64),
 'site_5114_Mahape_Navi_Mumbai_MPCB_15Min': (5,
  89.458333     1
  442.500000    1
  452.541667    1
  523.166667    1
  842.375000    1
  Name: count, dtype: int64),
 'site_5678_RVCE-Mailasandra_Bengaluru_KSPCB_15Min': (2,
  736.958333     1
  1467.9583

In [42]:
# Calculate average number of gaps across all sites
avg_num_gaps = sum(total_gaps for total_gaps, _ in gaps_per_site.values()) / len(gaps_per_site)

# Calculate average sequence length across all sites
all_sequence_lengths = []
for site_name, (total_gaps, gap_counts) in gaps_per_site.items():
    all_sequence_lengths.extend(gap_counts.index.tolist())

avg_sequence_length = sum(all_sequence_lengths) / len(all_sequence_lengths)

# Create buckets for sequence lengths
seq_length_series = pd.Series(all_sequence_lengths)
bins = [0, 30, 90, 180, 365, 730, float('inf')]
labels = ['0-30 days', '30-90 days', '90-180 days', '180-365 days', '1-2 years', '2+ years']
seq_length_buckets = pd.cut(seq_length_series, bins=bins, labels=labels).value_counts().sort_index()

print(f"Average number of gaps per site: {avg_num_gaps:.2f}")
print(f"Average sequence length (days): {avg_sequence_length:.2f}")
print(f"\nSequence length distribution:")
print(seq_length_buckets)


Average number of gaps per site: 4.42
Average sequence length (days): 551.95

Sequence length distribution:
0-30 days       586
30-90 days      465
90-180 days     311
180-365 days    249
1-2 years       276
2+ years        546
Name: count, dtype: int64
